In [4]:
import requests
import time
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from random import randint

# Get Wikipedia article content

In [5]:
import wikipedia as wiki

In [7]:
#park_df = pd.DataFrame(columns=['Park', 'State', 'Category', 'Url'])
park_df_f = '/Users/klarnemann/Documents/Insight/Insight_project/data/scraped_parks.xlsx'
park_df = pd.read_excel(park_df_f, index_col=0)
park_df['Article_content']=''
park_df.head()

,Park,State,City,Street Address,Url,Category,Features,Description,Article_content
0,Bladon Springs State Park,Alabama,Silas,3921 Bladon Rd,https://www.stateparks.com/bladon_springs_stat...,State Park,"['Camping', 'Nearby Parks', 'Picnicking', 'Cur...","Centering around four mineral springs, this 3...",
1,Blue Springs State Park,Alabama,Clio,2595 AL-10,https://www.stateparks.com/blue_springs_state_...,State Park,"['Camping', 'Swimming', 'Fishing', 'Picnicking...","Fed by a crystal-clear underground spring, th...",
2,Chattahoochee State Park,Alabama,Gordon,250 Chattahoochee St Pk Rd,https://www.stateparks.com/chattahoochee_state...,State Park,"['Nearby Parks', 'Current Weather Prediction']",,
3,Cheaha State Park,Alabama,Delta,19644 AL-281,https://www.stateparks.com/cheaha_state_park_i...,State Park,"['History of the Area', 'Meeting Rooms', 'Cabi...","Located in Clay and Cleburne Counties, Cheaha...",
4,Chewacla State Park,Alabama,Auburn,124 Shell Toomer Pkwy,https://www.stateparks.com/chewacla_state_park...,State Park,"['Cabins', 'Camping', 'Swimming', 'Boating', '...",Chewacla State Park's 696 scenic acres are a ...,


In [8]:
for i, title in enumerate(park_df.Park):
    try:
        park_df.loc[i, 'Article_content'] = wiki.page(title).content
    except:
        continue
    if i % 100 == 0:
        print(i)
        park_df.to_excel(park_df_f)

0
100


/Users/klarnemann/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/klarnemann/anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


200
300
400
500
600
700
800
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2400
2500
2600
2800
2900
3000
3100
3200
3300
3400
3500
3600


In [13]:
park_df.to_excel(park_df_f)

In [9]:
park_df.head()

,Park,State,City,Street Address,Url,Category,Features,Description,Article_content
0,Bladon Springs State Park,Alabama,Silas,3921 Bladon Rd,https://www.stateparks.com/bladon_springs_stat...,State Park,"['Camping', 'Nearby Parks', 'Picnicking', 'Cur...","Centering around four mineral springs, this 3...",Bladon Springs State Park is a locally managed...
1,Blue Springs State Park,Alabama,Clio,2595 AL-10,https://www.stateparks.com/blue_springs_state_...,State Park,"['Camping', 'Swimming', 'Fishing', 'Picnicking...","Fed by a crystal-clear underground spring, th...",Blue Spring State Park is a state park located...
2,Chattahoochee State Park,Alabama,Gordon,250 Chattahoochee St Pk Rd,https://www.stateparks.com/chattahoochee_state...,State Park,"['Nearby Parks', 'Current Weather Prediction']",,"Chattahoochee State Park is a state-owned, pub..."
3,Cheaha State Park,Alabama,Delta,19644 AL-281,https://www.stateparks.com/cheaha_state_park_i...,State Park,"['History of the Area', 'Meeting Rooms', 'Cabi...","Located in Clay and Cleburne Counties, Cheaha...",Cheaha State Park is a publicly owned recreati...
4,Chewacla State Park,Alabama,Auburn,124 Shell Toomer Pkwy,https://www.stateparks.com/chewacla_state_park...,State Park,"['Cabins', 'Camping', 'Swimming', 'Boating', '...",Chewacla State Park's 696 scenic acres are a ...,Chewacla State Park is a publicly owned recrea...


In [12]:
park_df.shape

(3748, 9)

In [11]:
print(np.sum(park_df['Article_content'] == ''))
print(np.sum(park_df['Article_content'] != ''))

191
3557


# Scrape

In [2]:
main_url = 'https://www.stateparks.com/'

In [3]:
def scrape_directories(url, state, park_df, main_url=main_url):
    t0 = time.time()#start time
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    response_time = time.time() - t0
    time.sleep(10*response_time)
    outputs = []
    output_urls = []
    output_categories = []
    categories = []
    for i in soup.find_all(class_='parkType'):
        cat = i.string
        if cat[-1] == 's':
            cat = cat[:-1]
        categories.append(cat)
    for i in np.arange(len(soup.find_all('a'))):
        try:
            for cat in categories:
                val = soup.find_all('a')[i].string
                if cat in val:
                    outputs.append(val)
                    output_categories.append(cat)
                    output_url = soup.find_all('a')[i].get('href')
                    output_urls.append(main_url + output_url)
        except:
            continue
    n_parks = len(outputs)
    data = np.matrix([outputs, [state]*n_parks, output_categories, output_urls]).T
    park_df = park_df.append(pd.DataFrame(data, columns=park_df.columns), ignore_index=True)
    return park_df

In [4]:
def run_scrape_directories(state, park_df, main_url=main_url):
    state_fmt = state.lower()
    state_fmt = state_fmt.replace(' ', '_')
    state_url = '%s/%s_parks_and_recreation_destinations.html' % (main_url, state_fmt)
    print('Scrapping %s parks...' % (state))
    park_df = scrape_directories(state_url, state, park_df)
    return park_df

In [12]:
def parse_park_information(park_url, state):
    t0 = time.time()#start time
    page = requests.get(park_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    response_time = time.time() - t0
    state_str = ', %s' % (state)
    try:
        street_address, city = str(soup.find_all(class_='parkinfo')[0]).split(state_str)[0].split('<br/>')[-2:]
    except:
        street_address = ''
        city = ''
    try:
        headers = [item.string for item in soup.find_all(class_='psheader')]
    except:
        headers = []
    try:
        description = str(soup.find_all(class_='psbod'))
        description = re.sub('<.*?>', ' ' , description)
        description = description[1:-1]
    except:
        description = ''
    return [street_address, city, headers, description], response_time

In [41]:
def run_parse_park_information(park_index, park_df):
    state = park_df.loc[park_index, 'State']
    park_url = park_df.loc[park_index, 'Url']
    content, response_time = parse_park_information(park_url, state)
    park_df.loc[park_index, 'Street Address'] = content[0]
    park_df.loc[park_index, 'City'] = content[1]
    try:
        park_df.loc[park_index, 'Features'] = content[2]
    except:
        park_df['Features'] = park_df['Features'].astype(object)
        park_df.loc[park_index, 'Features']
    park_df.loc[park_index, 'Description'] = content[3]
    return park_df, response_time

# Run

In [7]:
#park_df = pd.DataFrame(columns=['Park', 'State', 'Category', 'Url'])
park_df_f = '/Users/klarnemann/Documents/Insight/Insight_project/data/scraped_parks.xlsx'
park_df = pd.read_excel(park_df_f, index_col=0)

In [36]:
park_df['Features'] = park_df['Features'].astype(object)

In [8]:
park_df.shape

(3748, 8)

In [9]:
park_df.head()

,Park,State,City,Street Address,Url,Category,Features,Description
0,Bladon Springs State Park,Alabama,NaN,NaN,https://www.stateparks.com/bladon_springs_stat...,State Park,NaN,NaN
1,Blue Springs State Park,Alabama,NaN,NaN,https://www.stateparks.com/blue_springs_state_...,State Park,NaN,NaN
2,Chattahoochee State Park,Alabama,NaN,NaN,https://www.stateparks.com/chattahoochee_state...,State Park,NaN,NaN
3,Cheaha State Park,Alabama,NaN,NaN,https://www.stateparks.com/cheaha_state_park_i...,State Park,NaN,NaN
4,Chewacla State Park,Alabama,NaN,NaN,https://www.stateparks.com/chewacla_state_park...,State Park,NaN,NaN


~~~
park_df['City'] = ''
park_df['Street Address'] = ''
park_df['Features'] = ''
park_df['Description'] = ''
park_df = park_df[['Park', 'State', 'City', 'Street Address', 'Url', 'Category', 'Features', 'Description']]
park_df.head()
~~~

In [15]:
park_url = park_df.loc[0, 'Url']
print(park_url)
tmp_content, tmp_time = parse_park_information(park_url, park_df.loc[0,'State'])

https://www.stateparks.com/bladon_springs_state_park_in_alabama.html


In [20]:
tmp_content[3]

' Centering around four mineral springs, this 357-acre park in Choctaw County in southwest Alabama provides modern campsites, shelters, tables and grills for daytime and nighttime family outings. The Park is located one mile north of Bladon Springs in southwest Alabama.  Opened as a spa in 1838, guests traveled from all over the country to benefit from the "curing" properties of the mineral springs containing sulfur, iron, magnesium, and calcium. A grand Greek Revival style mansion large enough to accommodate 200 people was built in 1846, making it one of the largest wooden hotels built in Alabama.  The "health" resort which gained the nickname "Saratoga of the South," stayed open through the Civil War until it closed after the turn of the century when the novelty of mineral springs wore off. The state purchased the property in 1934. ,  The park has a modern campground with 10 campsites. Each site has water, 30 amp electricity, sewer connections and a picnic table and grill. Restrooms 

In [42]:
tmp_df, response_time = run_parse_park_information(0, park_df)
tmp_df.loc[0]

Park                                      Bladon Springs State Park
State                                                       Alabama
City                                                          Silas
Street Address                                       3921 Bladon Rd
Url               https://www.stateparks.com/bladon_springs_stat...
Category                                                 State Park
Features          [Camping, Nearby Parks, Picnicking, Current We...
Description        Centering around four mineral springs, this 3...
Name: 0, dtype: object

In [58]:
#state_index = park_df.index[park_df['State'] == 'New York']
#state_index = park_df.index[park_df['State'] == 'New Jersey']
#state_index = park_df.index[park_df['State'] == 'Connecticut']
state_index = park_df.index[park_df['State'] == 'Pennsylvania']

In [59]:
for i, park_index in enumerate(state_index):
    if i % 25 == 0:
        print(i)
        park_df.to_excel(park_df_f)
    park_df, response_time = run_parse_park_information(park_index, park_df)
    time.sleep(10*response_time)
park_df.loc[state_index]

0
25
50
75
100
125


,Park,State,City,Street Address,Url,Category,Features,Description
2809,Archbald Pothole State Park,Pennsylvania,Archbald,960 Scranton Carbondale Hwy,https://www.stateparks.com/archbald_pothole_st...,State Park,"[History of the Area, Nearby Accommodations, N...",Archbald Pothole State Park is a 150-acre par...
2810,Bald Eagle State Park,Pennsylvania,Howard,149 Main Park Rd,https://www.stateparks.com/bald_eagle_state_pa...,State Park,"[History of the Area, Camping, Marinas, Swimmi...","The 5,900-acre park lies in the broad Bald Ea..."
2811,Beltzville State Park,Pennsylvania,Lehighton,2950 Pohopoco Dr.,https://www.stateparks.com/beltzville_state_pa...,State Park,"[Swimming, Boating, Nearby Parks, Picnicking, ...","In Carbon County, the 2,972-acre Beltzville S..."
2812,Bendigo State Park,Pennsylvania,St Marys,533 Johnsonburg Rd,https://www.stateparks.com/bendigo_state_park_...,State Park,"[History of the Area, Nearby Parks, Picnicking...",The 100-acre Bendigo State Park is in a small...
2813,Big Pocono State Park,Pennsylvania,Tannersville,980 Camelback Rd,https://www.stateparks.com/big_pocono_state_pa...,State Park,"[History of the Area, Nearby Accommodations, N...",Big Pocono State Park is in Monroe County in ...
2814,Big Spring State Park,Pennsylvania,Blain,"RR 1, Box 486",https://www.stateparks.com/big_spring_state_pa...,State Park,"[History of the Area, Nearby Parks, Picnicking...","Tucked in the side of Conococheague Mountain,..."
2815,Black Moshannon State Park,Pennsylvania,Philipsburg,4216 Beaver Rd,https://www.stateparks.com/black_moshannon_sta...,State Park,"[Cabins, Camping, Swimming, Boating, Nearby Pa...","Perched on the top of the Allegheny Plateau, ..."
2816,Blue Knob State Park,Pennsylvania,Imler,Park Rd,https://www.stateparks.com/blue_knob_state_par...,State Park,"[History of the Area, Camping, Swimming, Nearb...",Blue Knob State Park offers year-round wilder...
2817,Brandywine Battlefield State Park,Pennsylvania,,,https://www.stateparks.com/brandywine_battlefi...,State Park,"[Nearby Parks, Area Campgrounds, Area Fishing ...",
2818,Bushy Run Battlefield State Park,Pennsylvania,,,https://www.stateparks.com/bushy_run_battlefie...,State Park,"[Nearby Parks, Area Campgrounds, Area Fishing ...",


In [60]:
states = np.loadtxt('/Users/klarnemann/Documents/Insight/Insight_project/data/states.txt', delimiter='\t', dtype='str')
states

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype='<U14')

In [62]:
missing_states = list(set.difference(set(states), set(['New York', 'New Jersey', 'Connecticut', 'Pennsylvania'])))
missing_states

['South Dakota',
 'Louisiana',
 'New Hampshire',
 'Kansas',
 'North Carolina',
 'Rhode Island',
 'Indiana',
 'Arkansas',
 'Oklahoma',
 'Colorado',
 'Virginia',
 'Minnesota',
 'Michigan',
 'Delaware',
 'South Carolina',
 'Tennessee',
 'Florida',
 'Washington',
 'Wyoming',
 'Nebraska',
 'Utah',
 'Illinois',
 'Hawaii',
 'Maryland',
 'Montana',
 'New Mexico',
 'Maine',
 'West Virginia',
 'Wisconsin',
 'Mississippi',
 'Kentucky',
 'Arizona',
 'Nevada',
 'Alaska',
 'Missouri',
 'Alabama',
 'North Dakota',
 'Massachusetts',
 'Ohio',
 'Oregon',
 'Texas',
 'Vermont',
 'Idaho',
 'Iowa',
 'California',
 'Georgia']

In [64]:
for state in missing_states:
    print(state)
    state_index = park_df.index[park_df['State'] == state]
    for i, park_index in enumerate(state_index):
        if i % 25 == 0:
            print(i)
            park_df.to_excel(park_df_f)
        park_df, response_time = run_parse_park_information(park_index, park_df)
        time.sleep(10*response_time)

South Dakota
0
25
Louisiana
0
25
New Hampshire
0
25
Kansas
0
25
North Carolina
0
25
50
Rhode Island
0
Indiana
0
25
50
Arkansas
0
25
50
Oklahoma
0
25
50
75
Colorado
0
25
Virginia
0
25
50
Minnesota
0
25
50
75
Michigan
0
25
50
75
100
Delaware
0
25
South Carolina
0
25
50
Tennessee
0
25
50
Florida
0
25
50
75
100
125
150
175
Washington
0
25
50
75
100
125
150
175
200
Wyoming
0
25
Nebraska
0
25
Utah
0
25
50
Illinois
0
25
50
75
100
Hawaii
0
Maryland
0
25
50
Montana
0
25
50
75
New Mexico
0
25
50
Maine
0
25
50
West Virginia
0
25
50
Wisconsin
0
25
50
Mississippi
0
25
Kentucky
0
25
50
Arizona
0
25
50
Nevada
0
25
50
Alaska
0
25
Missouri
0
25
50
Alabama
0
25
50
North Dakota
0
25
Massachusetts
0
25
50
75
Ohio
0
25
50
75
100
125
Oregon
0
25
50
75
100
125
150
175
Texas
0
25
50
75
Vermont
0
25
Idaho
0
25
50
Iowa
0
25
50
75
California
0
25
50
75
100
125
150
175
200
225
Georgia
0
25
50
75


In [65]:
import randorandint_rows, n_cols = park_df.shape
rows = [random.randint(1,n_rows) for i in range(10)]
park_df.loc[rows]

,Park,State,City,Street Address,Url,Category,Features,Description
3461,Lewis and Clark Trail State Park,Washington,Phone: 509-337-6457\r\n</div>\n</div>,"<div class=""parkinfo"" style="" display: flex; ...",https://www.stateparks.com/lewis_and_clark_tra...,State Park,"[Nature of the Area, History of the Area, Camp...",Lewis and Clark Trail State Park is a 37-acre...
332,Tahoe State Park,California,Phone: 530-583-3074\r\n</div>\n</div>,"<div class=""parkinfo"" style="" display: flex; ...",https://www.stateparks.com/tahoe_state_park_in...,State Park,"[Nearby Parks, Area Campgrounds, Current Weath...","Lake Tahoe State Park, situated on the und..."
747,Gamble Plantation Historic State Park,Florida,Ellenton,3708 Patten Avenue,https://www.stateparks.com/gamble_plantation_h...,State Park,"[Nearby Parks, Area Campgrounds, Current Weath...",This antebellum mansion was home to Major Rob...
450,Santa Monica Mountains National Recreation Area,California,Thousand Oaks,Santa Monica Mountains National Recreation Area,https://www.stateparks.com/santa_monica_mounta...,National Recreation Area,"[Camping, Nearby Parks, Nature Programs, Area ...",Santa Monica Mountains rise above Los Angeles...
2613,Wichita Mountains National Wildlife Refuge Hea...,Oklahoma,,,https://www.stateparks.com/wichita_mountains_n...,National Wildlife Refuge,"[Nearby Parks, Current Weather Prediction]",
1940,War Horse National Wildlife Refuge,Montana,Lewistown,Airport Road,https://www.stateparks.com/war_horse_national_...,National Wildlife Refuge,"[Current Weather Prediction, Directions]",Extending 125 miles up the Missouri River fro...
375,Point Lobos State Reserve,California,Carmel,Route 1 Box 62,https://www.stateparks.com/point_lobos_state_r...,State Reserve,"[Nearby Parks, Area Attractions, Area Campgrou...","This area contains headlands, coves and rolli..."
3326,Claytor Lake State Park,Virginia,Dublin,6620 Ben H. Bolen Drive,https://www.stateparks.com/claytor_lake_state_...,State Park,"[Swimming, Boating, Fishing, Nearby Parks, Pic...","Located on the 4,500 acre, 21-mile long Clayt..."
742,Dade Battlefield Historic State Park,Florida,Bushnell,7200 CR 603 South Battlefield Drive,https://www.stateparks.com/dade_battlefield_hi...,State Park,"[History of the Area, Visitors Center, Nearby ...",The battle that started the Second Seminole W...
1824,Lake Wappapello State Park,Missouri,Williamsville,"HC 2, Box 102",https://www.stateparks.com/lake_wappapello_sta...,State Park,"[Cabins, Camping, Nearby Accommodations, Swimm...","Nestled in the southeast region of Missouri,..."


~~~
for state in states:
    park_df = run_scrape_content(state, park_df, main_url=main_url)
    park_df.to_excel('/Users/klarnemann/Documents/Insight/Insight_project/data/scraped_parks.xlsx')
~~~

# Explore

In [67]:
state_url = 'https://www.stateparks.com/new_jersey_parks_and_recreation_destinations.html'

In [72]:
state = 'New Jersey'
print(state.lower())
print(state.replace(' ', '_'))

new jersey
New_Jersey


In [46]:
page = requests.get(state_url)
soup = BeautifulSoup(page.text, 'html.parser')

In [47]:
categories = []
for i in soup.find_all(class_='parkType'):
    cat = i.string
    if cat[-1] == 's':
        cat = cat[:-1]
    categories.append(cat)
categories

['State Park',
 'State Forest',
 'State Airport',
 'State Wildlife Management Area',
 'National Park',
 'National Historic Site',
 'National Historic Park',
 'National Wildlife Refuge',
 'National Recreation Area']

In [25]:
for cat in categories:
    if cat in soup.find_all('a')[8]


<a href="//www.naturallyamazing.com/americasparks/16988.jpg" rel="prettyPhoto[gallery2]" title="Point Reyes National Seashore &lt;span style=font-style:italic;&gt;McClures Beach&lt;/span&gt; © &lt;a style=\'color:inherit\' href=/scripts/photographer.asp?pid=10892 target=\'photographer\'&gt;Rob Nuti&lt;/a&gt;&lt;br&gt;Wave crash looking south from McClures Beach">...</a>

In [43]:
soup.find_all('a')[21].get('href')

'ahjumawi_lava_springs_state_park_in_california.html'

In [164]:
#park_url = 'https://www.stateparks.com/stephens_state_park_in_new_jersey.html'
park_url = 'https://www.stateparks.com/wharton_state_forest_in_new_jersey.html'

In [165]:
page = requests.get(park_url)
soup = BeautifulSoup(page.text, 'html.parser')

In [166]:
parse_park_information(park_url, 'New Jersey')

(['31 Batsto Road',
  'Hammonton',
  ['History of the Area',
   'Cabins',
   'Camping',
   'Boating',
   'Nearby Parks',
   'Trails',
   'Area Attractions',
   'Area Campgrounds',
   'Current Weather Prediction',
   'Directions'],
  " Throughout Wharton are rivers and streams for canoeing, hiking trails (including a major section of the Batona Trail), miles of unpaved roads for mountain biking and horseback riding and numerous lakes, ponds and fields ideal for wildlife observation. Bald eagles, red-tailed hawks, marsh hawks, ospreys, great blue herons, swans, screech owls, great-horned owls, bluebirds, hummingbirds, purple martins, goldfinch, turkeys, beavers, river otters, fox and deer are only some of the wildlife the alert visitor can see. ,  Wharton State Forest is the largest single tract of land within the New Jersey State Park System. It is also the site of Batsto Village, a former bog iron and glassmaking industrial center from 1766 to 1867 that currently reflects the agricultu

In [146]:
soup.find_all(class_='psheader')[0].string

'History of the Area'

In [135]:
str(soup.find_all(class_='parkinfo')[0]).split(', New Jersey')[0].split('<br/>')[-2:]

['800 Willow Grove St.', 'Hackettstown']

In [107]:
soup.find_all(class_='parkinfo')[0].find_all('a')

[<a href="http://www.nj.gov/dep/parksandforests/parks/stephens.html" target="parkURL">STEPHENS STATE PARK</a>]

In [142]:
str(soup.find_all(class_='psbod'))

'[<div class="psbod">The Musconetcong River, an excellent freshwater fishing spot, flows over and around boulders through Stephens State Park. An easy walking, but rocky path follows the river with places for picnicking in shaded areas.</div>, <div class="psbod">Less than one mile east along Waterloo Road at Saxton Falls are the remains of one of the Morris Canal?s twenty-three locks and a section of the old canal?s towpath. The original canal was 90 miles in length, stretching from Newark to Phillipsburg, and was used to ship coal and iron across the state. However, the canal could not compete with the development of the Morris and Essex Railroad in the late 1800s, and the canal was eventually abandoned.</div>, <div class="psbod">Campsites: Forty tent and small trailer sites with fire rings and picnic tables. Flush toilets are within walking distance. Open April 1 through October 31. </div>, <div class="psbod">Electric motors and gas motors boats are allowed to be on the Musconetcong 

In [142]:
str(soup.find_all(class_='psbod'))

'[<div class="psbod">The Musconetcong River, an excellent freshwater fishing spot, flows over and around boulders through Stephens State Park. An easy walking, but rocky path follows the river with places for picnicking in shaded areas.</div>, <div class="psbod">Less than one mile east along Waterloo Road at Saxton Falls are the remains of one of the Morris Canal?s twenty-three locks and a section of the old canal?s towpath. The original canal was 90 miles in length, stretching from Newark to Phillipsburg, and was used to ship coal and iron across the state. However, the canal could not compete with the development of the Morris and Essex Railroad in the late 1800s, and the canal was eventually abandoned.</div>, <div class="psbod">Campsites: Forty tent and small trailer sites with fire rings and picnic tables. Flush toilets are within walking distance. Open April 1 through October 31. </div>, <div class="psbod">Electric motors and gas motors boats are allowed to be on the Musconetcong 

In [141]:
re.sub('<.*?>', ' ' , str(soup.find_all(class_='psbod')))

"[ The Musconetcong River, an excellent freshwater fishing spot, flows over and around boulders through Stephens State Park. An easy walking, but rocky path follows the river with places for picnicking in shaded areas. ,  Less than one mile east along Waterloo Road at Saxton Falls are the remains of one of the Morris Canal?s twenty-three locks and a section of the old canal?s towpath. The original canal was 90 miles in length, stretching from Newark to Phillipsburg, and was used to ship coal and iron across the state. However, the canal could not compete with the development of the Morris and Essex Railroad in the late 1800s, and the canal was eventually abandoned. ,  Campsites: Forty tent and small trailer sites with fire rings and picnic tables. Flush toilets are within walking distance. Open April 1 through October 31.  ,  Electric motors and gas motors boats are allowed to be on the Musconetcong River. Each person must have on a wearable Coast Guard approved Personal Floatation Dev

In [137]:
soup.find_all(class_='psbod')[0]

<div class="psbod">The Musconetcong River, an excellent freshwater fishing spot, flows over and around boulders through Stephens State Park. An easy walking, but rocky path follows the river with places for picnicking in shaded areas.</div>